In [2]:
# imports
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [35]:
# crawl dealbok companies page
url = 'https://dealbook.co/companies'

# get table data
r = requests.get(url)
html = r.text

# parse html
soup = BeautifulSoup(html, 'html.parser')

In [34]:
# find number of pages
pages = soup.find('ul', attrs={'class':'pagination'})

# find "Last page" link
last_page = pages.find_all('li')[-1].find('a')['href']

# get last page number
last_page_num = int(last_page.split('=')[-1])


In [54]:
# get all company from current page
def get_companies(soup):
    # get table
    table = soup.find('table', attrs={'class':'table table-hover'})
    # get table rows
    rows = table.find_all('tr')
    
    # get table data and append headers
    data = []
    for row in rows[1:]:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols])

    return data

In [70]:
# get all companies from all pages
# for page in range(1, last_page_num + 1):
companies = []
for page in range(1, 5):
    # get url
    url = 'https://dealbook.co/companies?page=' + str(page)
    # get table data
    r = requests.get(url)
    html = r.text
    soup = BeautifulSoup(html, 'html.parser')
    data = get_companies(soup)
    
    # append data
    companies.extend(data)
    print('page ' + str(page) + ' done')

page 1 done
page 2 done
page 3 done
page 4 done


In [71]:
# remove first and last list item for each row
for row in companies:
    row.pop(0)
    row.pop(-1)


In [72]:
# create dataframe
columns = ['Company', 'Status', 'Location', 'Deals', 'Total Raised']
df = pd.DataFrame(companies, columns=columns)

In [77]:
# save to csv
df.to_csv('../data/01_raw/dealbook_companies.csv', index=False, sep=';')